# Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

- 🤝 Breakout Room #1
  1. Task 1: Installing Required Libraries
  2. Task 2: Set Environment Variables
  3. Task 3: Synthetic Dataset Generation for Evaluation using Ragas
  4. Task 4: Evaluating our Pipeline with Ragas
  5. Task 6: Making Adjustments and Re-Evaluating

But first! Let's set some dependencies!

## Dependencies and API Keys:

> NOTE: Please skip the pip install commands if you are running the notebook locally.

In [1]:
#!pip install -qU ragas==0.2.10

In [3]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

We'll also need to provide our API keys.

First, OpenAI's for our LLM/embedding model combination!

In [1]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

OPTIONALLY:

We can also provide a Ragas API key - which you can sign-up for [here](https://app.ragas.io/).

In [2]:
os.environ["RAGAS_APP_TOKEN"] = getpass("Please enter your Ragas API key!")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [1]:
!mkdir data

mkdir: data: File exists


In [2]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: simonwillison.net


In [3]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: simonwillison.net


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [4]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [5]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [8]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [6]:
dataset.to_pandas()

NameError: name 'dataset' is not defined

#### OPTIONAL:

If you've provided your Ragas API key - you can use this web interface to look at the created data!

In [10]:
dataset.upload()

Testset uploaded! View at https://app.ragas.io/dashboard/alignment/testset/edd91e3b-b4df-4dd5-8b0a-0054d0da36b5


'https://app.ragas.io/dashboard/alignment/testset/edd91e3b-b4df-4dd5-8b0a-0054d0da36b5'

## LangChain RAG

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [11]:
path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

Now that we have our data loaded, let's split it into chunks!

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

75

#### ❓ Question: 

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?

answer : it lets us to keep some of the context persist through chunks and help LLM get sequential chunks fetch if necessary.

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [13]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [14]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="ai_across_years",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="ai_across_years",
    embedding=embeddings,
)

/Users/ugurcekmez/Desktop/workspace/ai-makerspace/AIE6/08_Evaluating_RAG_With_Ragas/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:758: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is empty, alternative syntax for `is_empty: \&quot;field_name\&quot;`",
/Users/ugurcekmez/Desktop/workspace/ai-makerspace/AIE6/08_Evaluating_RAG_With_Ragas/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:762: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is null, alternative syntax for `is_null: \&quot;field_name\&quot;`",


We can now add our documents to our vector store.

In [15]:
_ = vector_store.add_documents(documents=split_documents)

Let's define our retriever.

In [16]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

Now we can produce a node for retrieval!

In [17]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### Augmented

Let's create a simple RAG prompt!

In [18]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-mini` to avoid using the same model as our judge model.

In [19]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Then we can create a `generate` node!

In [20]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [21]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [22]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [23]:
response = graph.invoke({"question" : "How are LLM agents useful?"})

In [24]:
response["response"]

'LLM agents are useful in several ways, as highlighted in the provided context:\n\n1. **Automation of Tasks**: They can act on your behalf, similar to a travel agent, by automating various tasks and processes. This can involve providing assistance in research or managing digital environments.\n\n2. **Problem-Solving Capabilities**: LLMs can be designed to access tools and run processes in a loop to solve specific problems. This capability enhances their utility in various applications.\n\n3. **Ease of Development**: The surprising ease of building LLMs with relatively minimal code (a few hundred lines of Python) means that they can be developed by a wider range of people, not just those with extensive resources.\n\n4. **Accessibility**: With advancements like the release of models that can run on personal devices, LLMs have become more accessible to individuals and smaller organizations, allowing for broader experimentation and use.\n\n5. **Iterative Learning**: LLMs can generate code 

## Evaluating the App with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [25]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [26]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,Wht iz Googles role in the development of LLMs?,[So training an LLM still isn’t something a ho...,[Code may be the best application The ethics o...,Google is one of the organizations that has pr...,Google is one of the organizations that have p...,single_hop_specifc_query_synthesizer
1,"Do LLMs get lazy in Decembur, and how does the...",[MLC Chat: Llama - [System] Ready to chat. a p...,[Based Development As a computer scientist and...,The provided context does not indicate that LL...,The honest answer is 'maybe'! No-one is entire...,single_hop_specifc_query_synthesizer
2,What Simon Willison say about weblog and AI in...,[Simon Willison’s Weblog\n\nSubscribe\n\nStuff...,[Simon Willison’s Weblog Subscribe Stuff we fi...,"In 2023, Simon Willison highlighted that Large...",Simon Willison’s Weblog mentions that 2023 was...,single_hop_specifc_query_synthesizer
3,What are the implications of GPT-4V on AI ethics?,[This is a huge advantage for open over closed...,[easy to follow. The rest of the document incl...,The implications of GPT-4V on AI ethics are mu...,The context discusses the ethical implications...,single_hop_specifc_query_synthesizer
4,How did the breaking of the GPT-4 barrier and ...,"[LLM prices crashed, thanks to competition and...",[<1-hop>\n\nPrompt driven app generation is a ...,The breaking of the GPT-4 barrier and the subs...,"In 2024, the breaking of the GPT-4 barrier and...",multi_hop_abstract_query_synthesizer
5,How did the breaking of the GPT-4 barrier and ...,"[LLM prices crashed, thanks to competition and...",[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, the breaking of the GPT-4 barrier and...","In 2024, the breaking of the GPT-4 barrier and...",multi_hop_abstract_query_synthesizer
6,How have AI model rankings and environmental i...,[OpenAI are not the only game in town here. Go...,[<1-hop>\n\nPrompt driven app generation is a ...,The evolution of AI model rankings and environ...,AI model rankings have seen significant change...,multi_hop_abstract_query_synthesizer
7,How have AI model rankings and environmental i...,[OpenAI are not the only game in town here. Go...,[<1-hop>\n\nPrompt driven app generation is a ...,The evolution of AI model rankings and environ...,AI model rankings have seen significant change...,multi_hop_abstract_query_synthesizer
8,How does the use of synthetic data in training...,[One of the best descriptions I’ve seen of thi...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,The use of synthetic data in training models l...,The use of synthetic data in training models l...,multi_hop_specific_query_synthesizer
9,How has the development of GPT-4 and GPT-4o mo...,[This is a huge advantage for open over closed...,[<1-hop>\n\nPrompt driven app generation is a ...,The development of GPT-4 and GPT-4o models has...,The development of GPT-4 and GPT-4o models has...,multi_hop_specific_query_synthesizer


Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [28]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [29]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

Next up - we simply evaluate on our desired metrics!

In [30]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

{'context_recall': 0.6236, 'faithfulness': 0.6895, 'factual_correctness': 0.4758, 'answer_relevancy': 0.9570, 'context_entity_recall': 0.3895, 'noise_sensitivity_relevant': 0.2723}

## Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!

> NOTE: This will be using Cohere's Rerank model (which was updated fairly [recently](https://docs.cohere.com/v2/changelog/rerank-v3.5)) - please be sure to [sign-up for an API key!](https://docs.cohere.com/reference/about)

In [31]:
os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

In [34]:
#!pip install -qU cohere langchain_cohere


We'll first set our retriever to return more documents, which will allow us to take advantage of the reranking.

In [32]:
retriever = vector_store.as_retriever(search_kwargs={"k": 20})

Reranking, or contextual compression, is a technique that uses a reranker to compress the retrieved documents into a smaller set of documents.

This is essentially a slower, more accurate form of semantic similarity that we use on a smaller subset of our documents.

In [33]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

We can simply rebuild our graph with the new retriever!

In [34]:
class State(TypedDict):
  question: str
  context: List[Document]
  response: str

graph_builder = StateGraph(State).add_sequence([retrieve_adjusted, generate])
graph_builder.add_edge(START, "retrieve_adjusted")
graph = graph_builder.compile()

In [35]:
response = graph.invoke({"question" : "How are LLM agents useful?"})
response["response"]

"LLM agents are considered useful primarily in two contexts: as digital assistants that can act on one's behalf (similar to a travel agent) and as tools that utilize LLMs in a loop to solve problems. They can be particularly effective in writing code, as programming languages have simpler grammatical structures compared to natural languages. However, there are significant concerns about their reliability and the challenge of gullibility, which can hinder their utility. Critics stress the need for better criticism and responsible usage of LLMs, highlighting the importance of addressing their limitations and potential negative impacts."

In [36]:
import time

for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(2) # To try to avoid rate limiting.

In [37]:
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

{'context_recall': 0.4804, 'faithfulness': 0.7021, 'factual_correctness': 0.4567, 'answer_relevancy': 0.9566, 'context_entity_recall': 0.3931, 'noise_sensitivity_relevant': 0.2585}

#### ❓ Question: 

Which system performed better, on what metrics, and why?

1st results : {'context_recall': 0.6236, 'faithfulness': 0.6895, 'factual_correctness': 0.4758, 'answer_relevancy': 0.9570, 'context_entity_recall': 0.3895, 'noise_sensitivity_relevant': 0.2723}

2nd results : {'context_recall': 0.4804, 'faithfulness': 0.7021, 'factual_correctness': 0.4567, 'answer_relevancy': 0.9566, 'context_entity_recall': 0.3931, 'noise_sensitivity_relevant': 0.2585}
